In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import sounddevice as sd
%matplotlib qt

In [22]:
# Step 1: Define the parameters

# Number of microphones
M = 3

# Order of Ambisonics
P = 4  

triangle_side = 4.2

# Constants for the attenuation function
volume_threshold = 0.9
volume_range = 0.9
hoa_threshold = 0.9
hoa_range = 1.3

def attenuation_and_rebalancing(d_m, t_l=volume_threshold, t_k=hoa_range, s_k_0=1.0, s_k_p_neg=-1.0, s_l_neg=-1.0):
    """
    Distance-dependent attenuation and component re-balancing function.

    Parameters:
    - d_m: distance to the m-th microphone
    - t_l: attenuation threshold
    - t_k: re-balancing threshold
    - s_k_0: slope for the 0th order component
    - s_k_p_neg: slope for higher-order components (p>0)
    - s_l_neg: slope for overall gain

    Returns:
    - Attenuation coefficient for the given distance
    """
    def l(d_m):
        return np.where(d_m <= t_l, 0, s_l(d_m - t_l))

    def k_p(d_m):
        return np.where(d_m <= t_k, 0, s_k_p(d_m - t_k))

    def s_l(delta_d):
        # Adjust this function based on the specific behavior of s_l
        # For example, you can use a linear function: return s_l_neg * delta_d
        return s_l_neg * delta_d

    def s_k_p(delta_d):
        # Adjust this function based on the specific behavior of s_k_p
        # For p=0, the slope is positive; for p>0, the slope is negative
        return np.where(delta_d <= 0, s_k_0, s_k_p_neg * delta_d)

    return 10 ** ((l(d_m) + k_p(d_m)) / 20.0)


def compute_distance(interp_point, mic_number, triangle_side):
    # Microphone positions in the equilateral triangle
    mic_positions = np.zeros((3, 2))
    
    mic_positions[0, :] = [0, 0]
    mic_positions[1, :] = [triangle_side, 0]
    mic_positions[2, :] = [triangle_side / 2, (triangle_side * np.sqrt(3)) / 2]
    
    
    # Euclidean distance between the interpolation point and the microphone
    distance = np.linalg.norm(interp_point - mic_positions[mic_number, :])
    
    return distance

In [14]:
import numpy as np
import scipy.signal
from scipy.io import wavfile

fs, mic1_signal = wavfile.read('/Users/prerna/Documents/spatial_audio/data/11_20/mic1.wav')
_, mic2_signal = wavfile.read('/Users/prerna/Documents/spatial_audio/data/11_20/mic2.wav')
_, mic3_signal = wavfile.read('/Users/prerna/Documents/spatial_audio/data/11_20/mic3.wav')

# Generate a 16 kHz tone
tone_freq = 16000
tone_duration = 5  # seconds
t = np.arange(0, tone_duration, 1/fs)
sync_tone = np.sin(2 * np.pi * tone_freq * t)

channel = 0

mic1_signal = mic1_signal[:, channel].astype(float)
mic2_signal = mic2_signal[:, channel].astype(float)
mic3_signal = mic3_signal[:, channel].astype(float)


min_length = min(len(mic1_signal), len(mic2_signal), len(mic3_signal))
mic1_signal = mic1_signal[:min_length]
mic2_signal = mic2_signal[:min_length]
mic3_signal = mic3_signal[:min_length]

# Perform the Fourier transform
f_mic1 = np.fft.fft(mic1_signal)
f_mic2 = np.fft.fft(mic2_signal)
f_mic3 = np.fft.fft(mic3_signal)

# Calculate cross-correlation in the frequency domain
correlation12 = np.fft.ifft(f_mic1 * np.conj(f_mic2))
correlation13 = np.fft.ifft(f_mic1 * np.conj(f_mic3))
correlation23 = np.fft.ifft(f_mic2 * np.conj(f_mic3))

# Find the lag with maximum correlation
lag12 = np.argmax(np.abs(correlation12))
lag13 = np.argmax(np.abs(correlation13))
lag23 = np.argmax(np.abs(correlation23))

# Calculate time delays
delay12 = lag12 / fs
delay13 = lag13 / fs
delay23 = lag23 / fs

print(f'Time delay between mic1 and mic2: {delay12:.5f} seconds')
print(f'Time delay between mic1 and mic3: {delay13:.5f} seconds')
print(f'Time delay between mic2 and mic3: {delay23:.5f} seconds')

lag = np.array([lag13, lag23, 0])

/var/folders/fr/rdj3hsr55n349b9r69qgjmch0000gn/T/ipykernel_6354/253236692.py:5: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, mic1_signal = wavfile.read('/Users/prerna/Documents/spatial_audio/data/11_20/mic1.wav')
/var/folders/fr/rdj3hsr55n349b9r69qgjmch0000gn/T/ipykernel_6354/253236692.py:6: WavFileWarning: Chunk (non-data) not understood, skipping it.
  _, mic2_signal = wavfile.read('/Users/prerna/Documents/spatial_audio/data/11_20/mic2.wav')
/var/folders/fr/rdj3hsr55n349b9r69qgjmch0000gn/T/ipykernel_6354/253236692.py:7: WavFileWarning: Chunk (non-data) not understood, skipping it.
  _, mic3_signal = wavfile.read('/Users/prerna/Documents/spatial_audio/data/11_20/mic3.wav')


Time delay between mic1 and mic2: 24.19179 seconds
Time delay between mic1 and mic3: 30.12671 seconds
Time delay between mic2 and mic3: 5.93433 seconds


In [15]:
# Step 2: Load your ambisonic signals y_m_p(n) from WAV files
path_to_files = '/Users/prerna/Documents/spatial_audio/data/11_20/'
file_names = ['mic1.wav', 'mic2.wav', 'mic3.wav']  # Adjust with your file names

# Initialize an array to store ambisonic signals
max_samples = max([wavfile.read(path_to_files + file_name)[1].shape[0] for file_name in file_names])
y_m_p = np.zeros((M, P, max_samples))  # Initialize with the maximum number of samples


for m in range(M):
    # Read the WAV file
    fs, y = wavfile.read(path_to_files + file_names[m])
    
    y = y[lag[m]: , :]
    
    # Take the 4 channels 
    y_m_p[m, :, :y.shape[0]] = y[:, :P].T

# Step 3: Implement the interpolation algorithm

# Number of samples
N = y_m_p.shape[2]

# Initialize the interpolated signal
x_p = np.zeros(N)

/var/folders/fr/rdj3hsr55n349b9r69qgjmch0000gn/T/ipykernel_6354/1698702362.py:6: WavFileWarning: Chunk (non-data) not understood, skipping it.
  max_samples = max([wavfile.read(path_to_files + file_name)[1].shape[0] for file_name in file_names])
/var/folders/fr/rdj3hsr55n349b9r69qgjmch0000gn/T/ipykernel_6354/1698702362.py:12: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, y = wavfile.read(path_to_files + file_names[m])


In [23]:
# Interpolation point coordinates
# interp_point = np.array([triangle_side / 2, (triangle_side * np.sqrt(3)) / 6])  # centroid
interp_point =  [1.05, (triangle_side * np.sqrt(3)) / 6]

# Initialize an array to store the attenuation coefficients for each microphone
a_p_values = np.zeros((M, N))

# Compute the distance for all samples at once
distances = np.array([compute_distance(interp_point, m, triangle_side) for m in range(M)])

# Compute the attenuation coefficients for each microphone using broadcasting
a_p_values = attenuation_and_rebalancing(distances[:, np.newaxis])

# Update the interpolated signal using vectorized NumPy operations
x_p = np.sum(a_p_values[:, :, np.newaxis] * y_m_p, axis=(0, 1))
x_p = x_p/1000000000.0

# Plot
time_axis = np.arange(1, N + 1)
wavfile.write('output_median1.wav', fs, x_p)
plt.plot(time_axis, x_p)
plt.title('Interpolated Signal')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.show()